In [ ]:
import os, sys
# Setup your root path
package_path = '/Users/miguel.romero/Workspace/globant-projects/quizzler/engine-quizz'
os.chdir(package_path)
if package_path not in sys.path:
  sys.path.append(package_path)
!pip3 install -r ./libs/requirements.txt

In [ ]:
!pip3 install pandas matplotlib

# AI Evaluator

The intent for this experiment is the hability to create a warmup given a context

In [ ]:
import pandas as pd

talent_model = pd.read_csv('./experiments/Talent-Model.csv')
talent_model.fillna(method='ffill', inplace=True)

## Analyzing skills for Seniority Level 4

The next analysis takes the skills for a Dev Level 4 (Senior) and just taking in consideration the skills metric from "Can perform with limited supervion" to "Can teach others"

In [ ]:
level4_tm = talent_model.iloc[:, [0,1,2,3,7]]
can_perform_w_limited_supervision = level4_tm.query('`Level 4` == "3 - Can perform with limited supervision"')
can_perform_w_supervision = level4_tm.query('`Level 4` == "4 - Can perform without supervision"')
can_teach = level4_tm.query('`Level 4` == "5 - Can teach others"')

In [ ]:
can_perform_w_limited_supervision.describe()

In [ ]:
can_perform_w_limited_supervision.groupby(by="Position’s Capabilities (SE)").count()

### Adding skill name

In [ ]:
temp_df_l3 = can_perform_w_limited_supervision.assign(One_Shot= lambda x: x['Subjects']+ " " + x['Tool / Framework / Libs'])
temp_df_l4 = can_perform_w_supervision.assign(One_Shot= lambda x: x['Subjects']+ " " + x['Tool / Framework / Libs'])
temp_df_l5 = can_teach.assign(One_Shot= lambda x: x['Subjects']+ " " + x['Tool / Framework / Libs'])

In [ ]:
temp_concat = pd.concat([temp_df_l3, temp_df_l4, temp_df_l5], axis=0)  # Concatenate the DataFrames
temp_concat = temp_concat.iloc[:, -1]
temp_concat


### Getting just the skill name

In [ ]:
senior_skills_l3 = temp_df_l3.iloc[:, -1]
senior_skills_l4 = temp_df_l4.iloc[:, -1]
senior_skills_l5 = temp_df_l5.iloc[:, -1]

In [ ]:
def split_to_pages(df, page_size):
  """
  Split a DataFrame into pages of a given size
  """
  return [df[i:i+page_size] for i in range(0, df.shape[0], page_size)]

In [ ]:
l3_pages = split_to_pages(senior_skills_l3, 10)
l4_pages = split_to_pages(senior_skills_l4, 10)
l5_pages = split_to_pages(senior_skills_l5, 10)

len(l3_pages), len(l4_pages), len(l5_pages)

In [ ]:
for list_items in l3_pages:
  for item in list_items:
    print(item)

In [ ]:
def generate_skills_prompt(skill_title: str, skills_set: list):
  """
  Generate a prompt for a set of skills
  """
  prompt_skills = f"{skill_title}\n"
  for skill in skills_set:
    prompt_skills += f"- {skill}\n"
  return prompt_skills

In [ ]:
l3_pages_prompts = [generate_skills_prompt("Can perform with limited supervision:", page) for page in l3_pages]
l4_pages_prompts = [generate_skills_prompt("Can perform without supervision:", page) for page in l4_pages]
l5_pages_prompts = [generate_skills_prompt("Can teach others:", page) for page in l5_pages]

In [ ]:
l3_pages_prompts[0]

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()
OPEN_API_KEY = os.getenv("OPEN_API_KEY")

llm = ChatOpenAI(
  openai_api_key=OPEN_API_KEY,
  model_name="gpt-3.5-turbo-0125",
  temperature=0)

In [ ]:
PROMPT_TEMPLATE = """
You are a software development technical interviews generator.
Your job is generate the QUESTIONS and CORRECT ANSWERS based on the SKILLS section.
Use the EVALUATION METRIC to generate QUESTIONS and ANSWERS according the skills level required.

EVALUATION METRIC:
- Can't perform: Candidate lacks the necessary skills or knowledge to fulfill the job requirements, even with guidance.
- Can perform with supervision: Candidate has basic understanding but needs constant guidance to perform the task effectively.
- Can perform with limited supervision: Candidate can perform the task independently but may occasionally require some guidance or clarification.
- Can perform without supervision: Candidate is fully capable of independently performing the task without any supervision or guidance.
- Can teach others: Candidate not only possesses the skills to perform the task but can effectively teach and transfer that knowledge to others.

SKILLS:
###
{skills}
###

Return the output in json format with the following structure:
[{{{{
  "question": <QUESTION>,
  "answer": <ANSWER>,
  "skill": <SKILL TO EVALUATE>,
}}}}]
"""

In [ ]:
# from langchain.chains.combine_documents import 
# create_stuff_documents_chain
import json
from operator import itemgetter
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template(PROMPT_TEMPLATE)

chain = {"skills": itemgetter("skills")} | prompt | llm | StrOutputParser() | (lambda x: json.loads(x))

result = chain.invoke({"skills": l3_pages_prompts[0]})

In [ ]:
class InterviewQuestionDto:
  def __init__(self, question: str, answer: str, skill: str):
    self.question = question
    self.answer = answer
    self.skill_name = skill

  @staticmethod
  def from_dict(data: list):
    return InterviewQuestionDto(data["question"], data["answer"], data["skill"])
  
  def set_dev_level(self, dev_level: str):
    self.dev_level = dev_level

  def set_skill_level(self, skill_level: str):
    self.skill_level = skill_level

class InterviewDto:
  def __init__(self, interview_title: str, questions: list):
    self.interview_title = interview_title
    self.questions = questions

  @staticmethod
  def from_questions(interview_title: str, data: list):
    questions = [InterviewQuestionDto.from_dict(q) for q in data]
    return InterviewDto(interview_title, questions)
  
  def enrich_questions(self, dev_level: str, skill_level: str):
    for q in self.questions:
      q.set_dev_level(dev_level)
      q.set_skill_level(skill_level)

  def add_questions(self, questions: list):
    self.questions.extend(questions)

  def to_dict(self):
    return {
      "interview_title": self.interview_title,
      "questions": [q.__dict__ for q in self.questions]
    }

  def __str__(self):
    return f"Skill: {self.skill_name} - Questions: {len(self.questions)}"

  def __repr__(self):
    return f"Skill: {self.skill_name} - Questions: {len(self.questions)}"

In [ ]:
interview = InterviewDto.from_questions("Senior - Skills level 3", result)
interview.enrich_questions("Senior", "Can perform with limited supervision")

In [ ]:
interview.to_dict()

In [ ]:
def write_json(file_name: str, skills_prompts: list):
  with open(f'{file_name}.json', 'w') as f:
    json.dump(skills_prompts, f)

In [ ]:
write_json('Dev Senior - Skills Level 3', interview.to_dict())